# RAS-Commander standard code cells 1-3: Install Packages and Prepare the Environment

In [1]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [2]:
# Import all required modules
from ras_commander import *  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from pathlib import Path
import sys
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Define versions to compare
versions = ['6.6', '6.5', '6.4.1', '6.3.1', '6.3', '6.2', "6.1", "6.0"] # NOTE: ras-commander does not support versions prior to 6.2 due to HDF5 file format changes

In [ ]:
# Extract BaldEagleCrkMulti2D project
project_path = RasExamples.extract_project(["BaldEagleCrkMulti2D"])

In [ ]:
# Init the ras_project with ras-commander to read all HEC-RAS project information 
init_ras_project(project_path, "6.5")
print(ras)
# If no ras object is defined in init_ras_project, it defaults to "ras" (useful for single project scripts)
# Display plan dataframe
ras.plan_df

In [ ]:
# Export Plan Numbers to List and Print
plan_numbers = ras.plan_df['plan_number'].tolist()
print(plan_numbers)


In [11]:
# Define run_simulation function for
import time
from ras_commander import RasGeo

def run_simulation(version, plan_number):
    # Initialize project for the specific version
    ras_project = init_ras_project(project_path, str(version))
    
    # Clear geometry preprocessor files for the plan
    plan_path = RasPlan.get_plan_path(plan_number, ras_object=ras_project)
    RasGeo.clear_geompre_files(plan_path, ras_object=ras_project)
    
    # Set the number of cores to 4
    RasPlan.set_num_cores(plan_number, "4", ras_object=ras_project)
    
    # Update plan run flags – setting "Run HTab" flag to 1 to force geometry preprocessing
    RasPlan.update_run_flags(plan_number, {"Run HTab": 1}, ras_object=ras_project)
    
    # Compute the plan
    start_time = time.time()
    success = RasCmdr.compute_plan(plan_number, ras_object=ras_project)
    total_time = time.time() - start_time
    
    if success:
        # Get the HDF file path for the plan results
        hdf_path = RasPlan.get_results_path(plan_number, ras_object=ras_project)
        
        # Extract runtime data from the HDF file
        runtime_data = HdfResultsPlan.get_runtime_data(hdf_path)
        
        # Extract required information from the runtime data
        preprocessor_time = runtime_data['Preprocessing Geometry (hr)'].values[0]
        unsteady_compute_time = runtime_data['Unsteady Flow Computations (hr)'].values[0]
        
        # Get volume accounting data from the HDF file
        volume_accounting = HdfResultsPlan.get_volume_accounting(hdf_path)
        # Extract Error Percent from the DataFrame
        volume_error = volume_accounting['Error Percent'].values[0] if not volume_accounting.empty else None
        
        # Print the extracted data
        print(f"\nExtracted Data for Plan {plan_number} in Version {version}:")
        print(f"Preprocessor Time: {preprocessor_time:.3f} hr")
        print(f"Unsteady Compute Time: {unsteady_compute_time:.3f} hr") 
        print(f"Volume Error: {volume_error:.3f}%" if volume_error is not None else "Volume Error: None")
        print(f"Total Time: {total_time/3600:.3f} hr\n")
        
        return {
            'Version': version,
            'Plan': plan_number,
            'Preprocessor Time (hr)': preprocessor_time,
            'Unsteady Compute Time (hr)': unsteady_compute_time,
            'Volume Error (%)': volume_error,
            'Total Time (hr)': total_time / 3600  # convert seconds to hours
        }
    else:
        return None

# OPTIONAL: Benchmark all plans in Version 6.6
Change the following cell from Markdown to Code and delete "```" to Run

```
#### Initialize results list
results = []
#### Loop through each plan number
for plan in plan_numbers:
    print(f"Running simulation for Version 6.6, Plan {plan}")
    result = run_simulation("6.6", plan)
    if result is not None:  #### Check if result is not None
        results.append(result)

#### Convert results list to DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv('hecras_plan_comparison.csv', index=False)

print("Results saved to 'hecras_plan_comparison.csv'")

#### Load and display the results dataframe
df = pd.read_csv('hecras_plan_comparison.csv')

#### Get plan titles from ras.plan_df and merge with results
plan_titles = pd.DataFrame({
    'Plan': ras.plan_df['plan_number'].str.zfill(2),  #### Ensure 2-digit format
    'Short Identifier': ras.plan_df['Short Identifier']
})
#### Convert df's Plan column to 2-digit string format
df['Plan'] = df['Plan'].astype(str).str.zfill(2)

df = df.merge(plan_titles, on='Plan', how='left')

print("Benchmarking Results:")
print(df.to_string(index=False))

#### Create a more comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

#### Function to create rotated labels
def plot_with_rotated_labels(ax, data, values, color, title, ylabel):
    bars = ax.bar(range(len(data)), values, color=color, alpha=0.7)
    ax.set_title(title, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    #### Set x-ticks at bar positions
    ax.set_xticks(range(len(data)))
    
    #### Create labels with plan numbers and titles
    labels = [f"Plan {plan}\n{title}" for plan, title in zip(data['Plan'], data['Short Identifier'])]
    ax.set_xticklabels(labels, rotation=45, ha='right')

#### Plot 1: Unsteady Runtime
plot_with_rotated_labels(ax1, df, df['Unsteady Compute Time (hr)'], 'blue', 
                        'Unsteady Runtime by Plan', 'Unsteady Runtime (hours)')

#### Plot 2: Volume Error  
plot_with_rotated_labels(ax2, df, df['Volume Error (%)'], 'red',
                        'Volume Error by Plan', 'Volume Error (%)')

#### Plot 3: Preprocessor Time
plot_with_rotated_labels(ax3, df, df['Preprocessor Time (hr)'], 'green',
                        'Preprocessor Time by Plan', 'Preprocessor Time (hours)')

#### Plot 4: Total Runtime
plot_with_rotated_labels(ax4, df, df['Total Time (hr)'], 'purple',
                        'Total Runtime by Plan', 'Total Runtime (hours)')

#### Adjust layout and display
plt.tight_layout(pad=3.0)
fig.suptitle('Plan Performance Comparison', fontsize=14, y=1.02)
plt.show()

#### Calculate and display summary statistics
print("\nSummary Statistics:")
print(df.describe())

#### Calculate plan-to-plan performance changes
print("\nPlan-to-Plan Performance Changes:")
df['Unsteady Runtime Change (%)'] = df['Unsteady Compute Time (hr)'].pct_change() * 100
print(df[['Plan', 'Short Identifier', 'Unsteady Runtime Change (%)']].to_string(index=False))
```

In [24]:
# Select the plan number you want to run across all versions
plan_number = '02'  # Make sure this is a string and include the leading zero


In [ ]:
# Run simulations for all versions with plan_number defined by user
results = []
for version in versions:
    print(f"Running simulation for Version {version}, Plan {plan_number}")
    result = run_simulation(version, plan_number) 
    if result is not None:  # Check if result is not None
        results.append(result)
        print(f"Completed: Version {version}, Plan {plan_number}")
    else:
        print(f"Failed: Version {version}, Plan {plan_number}")

# Create DataFrame from results
df = pd.DataFrame(results)

# Save initial results to CSV
df.to_csv('save_initial_results.csv', index=False)

print("Initial results saved to 'save_initial_results.csv'")


In [ ]:
# Create line graphs
plt.figure(figsize=(12, 6))

# Unsteady Runtime vs Version
plt.subplot(1, 2, 1)
# Convert Version to categorical type to handle string versions properly
plt.plot(pd.Categorical(df['Version']), df['Unsteady Compute Time (hr)'], marker='o')
plt.title(f'Unsteady Runtime vs HEC-RAS Version (Plan {plan_number})')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Unsteady Runtime (hours)')
plt.grid(True, linestyle='--', alpha=0.7)

# Volume Error vs Version
plt.subplot(1, 2, 2)
plt.plot(pd.Categorical(df['Version']), df['Volume Error (%)'], marker='o')
plt.title(f'Volume Error vs HEC-RAS Version (Plan {plan_number})')
plt.xlabel('HEC-RAS Version')
plt.ylabel('Volume Error (%)')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()